In [2]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [3]:
#Delete all objects in bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket('ayush-5-oct-24')
# Delete all objects in the bucket
for obj in bucket.objects.all():
    obj.delete()

In [4]:
arg_date = '2022-12-30'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'ayush-5-oct-24'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key_gen = 'Ayush_daily_report' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [5]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]

In [7]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16932\3669015963.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)


In [8]:
df_all = df_all.loc[:, columns]

In [9]:
df_all.dropna(inplace=True)

## Get opening price per ISIN and day

In [10]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

## Get closing price per ISIN and day

In [11]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

## Aggregations

In [12]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing

In [13]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [14]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [15]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [16]:
df_all = df_all.round(decimals=2)

In [17]:
df_all = df_all[df_all.Date >= arg_date]

In [18]:
key_gen

'Ayush_daily_report20241005_162711.parquet'

## Write to S3

In [19]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key_gen)

s3.Object(bucket_name='ayush-5-oct-24', key='Ayush_daily_report20241005_162711.parquet')

## Reading the uploaded file

In [20]:
prq_obj = bucket_target.Object(key=key_gen).get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.0
1,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.0
2,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205,0.0
3,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,0.0
4,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836,0.0
...,...,...,...,...,...,...,...,...
6459,XS2314660700,2022-12-31,22.26,21.92,21.92,22.28,0,0.0
6460,XS2376095068,2022-12-30,34.29,36.50,34.06,36.50,1000,0.0
6461,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.0
6462,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.0
